In [23]:
# 1. Installation des Bibliothèques
# !pip install langchain faiss-cpu openai tqdm python-dotenv

# 2. Importer les Bibliothèques et Charger les Variables d'Environnement
import os
import json
import time
import re
import faiss
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI, LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("La clé API OpenAI n'a pas été trouvée dans le fichier .env")

os.environ["OPENAI_API_KEY"] = openai_api_key

jsonl_path = 'data/decision_examples.jsonl'
sections_dir = 'data/sections'
output_directory = 'data/rules'

# Dossier dédié à l'index decision_index
index_dir = 'data/index/decision_index'
faiss_index_path = os.path.join(index_dir, 'decision_index.faiss')
docstore_path = os.path.join(index_dir, 'docstore.json')

if not os.path.exists(jsonl_path):
    raise FileNotFoundError(f"Le fichier {jsonl_path} n'a pas été trouvé.")

jsonl_mtime = os.path.getmtime(jsonl_path)
index_exists = os.path.exists(faiss_index_path) and os.path.exists(docstore_path)

recreate_index = False
if index_exists:
    faiss_mtime = os.path.getmtime(faiss_index_path)
    if jsonl_mtime > faiss_mtime:
        print("Le fichier JSONL a été modifié après la dernière mise à jour de l'index FAISS.")
        recreate_index = True
    else:
        print("L'index FAISS est à jour.")
else:
    print("L'index FAISS n'existe pas et sera créé.")
    recreate_index = True

def create_or_update_index():
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                obj = json.loads(line)
                data.append(obj)
    
    # Construire une liste de textes à partir de l'entrée
    texts = []
    for entry in data:
        if 'text' in entry:
            texts.append(entry['text'])
        elif 'messages' in entry and isinstance(entry['messages'], list):
            combined = "\n".join(msg["content"] for msg in entry["messages"] if "content" in msg)
            if combined.strip():
                texts.append(combined)

    if not texts:
        print("Aucun texte ni messages pour construire l'index.")
        return None, None, None

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=["\n\n", "\n", " ", ""]
    )
    
    all_chunks = []
    for t in texts:
        all_chunks.extend(text_splitter.split_text(t))
    
    embeddings = OpenAIEmbeddings()
    embedding_vectors = embeddings.embed_documents(all_chunks)
    if not embedding_vectors:
        print("Aucune embedding générée.")
        return None, None, None

    embedding_dim = len(embedding_vectors[0])
    
    index = faiss.IndexFlatL2(embedding_dim)
    index.add(np.array(embedding_vectors).astype('float32'))
    
    docstore = {str(i): chunk for i, chunk in enumerate(all_chunks)}
    
    os.makedirs(index_dir, exist_ok=True)
    faiss.write_index(index, faiss_index_path)
    print(f"Index FAISS sauvegardé dans {faiss_index_path}.")
    
    with open(docstore_path, 'w', encoding='utf-8') as f:
        json.dump(docstore, f)
    print(f"Docstore sauvegardé dans {docstore_path}.")
    
    return index, docstore, embeddings

if recreate_index:
    index, docstore, embeddings = create_or_update_index()
    if index is None:
        raise SystemExit("Impossible de créer l'index RAG (pas de texte).")
else:
    if not (os.path.exists(faiss_index_path) and os.path.exists(docstore_path)):
        print("Index ou docstore manquant, recréez l'index.")
        raise SystemExit
    index = faiss.read_index(faiss_index_path)
    with open(docstore_path, 'r', encoding='utf-8') as f:
        docstore = json.load(f)
    embeddings = OpenAIEmbeddings()
    print("Index FAISS et docstore chargés avec succès.")

def process_section(section_number: int):
    """
    Lis la section 'section_number' depuis data/sections/{section_number}.md
    Interroge le RAG pour obtenir des exemples ou des retours de joueurs, etc., 
    pour comprendre comment séparer l'histoire des règles ludiques.
    Les informations du RAG ne doivent pas être incluses dans le résultat final.
    La sortie finale doit contenir uniquement les règles spécifiques à cette section 
    (choix du joueur, stats, combats, énigmes...).
    """
    markdown_path = os.path.join(sections_dir, f"{section_number}.md")
    if not os.path.exists(markdown_path):
        print(f"La section {section_number} n'existe pas.")
        return

    with open(markdown_path, 'r', encoding='utf-8') as f:
        section_content = f.read()

    # Requête pour obtenir des instructions d'aide (RAG)
    query = ("Donne des exemples pour distinguer l'histoire du livre dont vous êtes le héros "
             "des règles, choix et décisions purement ludiques. Aide-moi à comprendre, mais "
             "je n'intégrerai pas ces exemples dans le résultat final.")
    query_embedding = embeddings.embed_query(query)
    query_vector = np.array([query_embedding]).astype('float32')

    k = 5
    distances, indices = index.search(query_vector, k)

    retrieved_texts = [docstore[str(i)] for i in indices[0] if str(i) in docstore]
    rag_instructions = "\n\n".join(retrieved_texts)

    prompt_template = """
Tu es un assistant chargé d'extraire les règles du jeu présentées dans une section d'un livre dont vous êtes le héros.
Le texte RAG ci-dessous fournit des exemples pour t'aider à comprendre comment séparer l'histoire des vraies règles du jeu.
Ne reprends pas ces exemples dans le résultat final, ils sont juste pour t'aider à comprendre.

RAG (exemples, ne pas les intégrer dans le résultat) :
{rag_instructions}

Section :
{section_content}

Instructions :
- Distingue l'histoire (description, narration) des éléments ludiques (choix du joueur, gain/perte de stats, argent, chemins à prendre, combat, énigme...).
- N'inclus que les règles et décisions spécifiques à cette section.
- Formate les règles en Markdown, par exemple :
  - Choix 1: ...
  - Endurance, CHance, Habileté +x si ...
  - Gagner, perdre de l'argent si
  - choisir cette action mène à ...
  - devoir se battre contre un monstre si
  - si je gagne une partie, je ...
  
Extrais uniquement ces éléments ludiques propres à cette section, sans reprendre les exemples du RAG.
"""
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["rag_instructions", "section_content"]
    )

    llm = ChatOpenAI(
        temperature=0,
        openai_api_key=openai_api_key,
        model_name="gpt-4o-mini"
    )
    chain = LLMChain(llm=llm, prompt=prompt)

    rules = chain.run(rag_instructions=rag_instructions, section_content=section_content)

    os.makedirs(output_directory, exist_ok=True)
    output_filename = f'section_{section_number}_rule.md'
    output_path = os.path.join(output_directory, output_filename)

    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(rules)

    print(f"Les règles de la section {section_number} ont été enregistrées dans {output_path}")

# Par exemple, traiter [1,48,164] , ou "all"
sections_to_process = "all"

if sections_to_process == "all":
    all_files = os.listdir(sections_dir)
    md_files = [f for f in all_files if f.endswith(".md")]
    all_sections = []
    for mf in md_files:
        m = re.search(r'(\d+)\.md', mf)
        if m:
            sec_num = int(m.group(1))
            all_sections.append(sec_num)
    sections_to_process = sorted(all_sections)
elif isinstance(sections_to_process, int):
    sections_to_process = [sections_to_process]

# Utilisation du multi-threading
# Nombre de threads = min(nombre de sections, 4) par exemple
from concurrent.futures import ThreadPoolExecutor, as_completed

num_threads = min(len(sections_to_process), 4)

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = {executor.submit(process_section, sec): sec for sec in sections_to_process}
    for future in as_completed(futures):
        sec = futures[future]
        try:
            future.result()
        except Exception as e:
            print(f"Erreur lors du traitement de la section {sec}: {e}")


L'index FAISS est à jour.
Index FAISS et docstore chargés avec succès.
Les règles de la section 1 ont été enregistrées dans data/rules\section_1_rule.md
Les règles de la section 4 ont été enregistrées dans data/rules\section_4_rule.md
Les règles de la section 2 ont été enregistrées dans data/rules\section_2_rule.md
Les règles de la section 3 ont été enregistrées dans data/rules\section_3_rule.md
Les règles de la section 5 ont été enregistrées dans data/rules\section_5_rule.md
Les règles de la section 7 ont été enregistrées dans data/rules\section_7_rule.md
Les règles de la section 6 ont été enregistrées dans data/rules\section_6_rule.md
Les règles de la section 8 ont été enregistrées dans data/rules\section_8_rule.md
Les règles de la section 9 ont été enregistrées dans data/rules\section_9_rule.md
Les règles de la section 10 ont été enregistrées dans data/rules\section_10_rule.md
Les règles de la section 11 ont été enregistrées dans data/rules\section_11_rule.md
Les règles de la sectio